<a href="https://colab.research.google.com/github/Darkdew/Web-scraper--handling-design-anomalies/blob/main/au_reachout_generic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
url = 'https://forums.au.reachout.com/t5/forums/searchpage/tab/message?advanced=false&allow_punctuation=false&q=chlamydia'

r = requests.get(url, headers= headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [65]:
full_list=[]

#counting number of available threads on page=n
threads= soup.find_all('a',{'class':'lia-link-navigation lia-page-link lia-user-name-link'})
number_of_threads=len(threads)

for count in range(number_of_threads):
            temp_list=[]
            # Extracting thread title
            thread_title_raw= soup.find_all('span',{'class': 'lia-message-unread lia-message-unread-windows'})[count].text
            thread_title= thread_title_raw.replace('\n','').replace('\t','')

            #Extracting Username
            threads1= soup.find_all('a',{'class':'lia-link-navigation lia-page-link lia-user-name-link'})
            username= threads1[count].find('span').text
            print(username)

            #Extracting_profile_url
            profile_url= threads1[count]['href']
            print(profile_url)

            
            r2=wd.get(profile_url)
            r3=wd.page_source#)  # results
            soup2 = BeautifulSoup(r3, 'html.parser')

            generics= soup2.find_all('div', {'class':'B3EUPNXoIzuOys-6RrYe9'})[0].find_all('li',{'class':'_3Bw5KIqCTKiMsOeIV9fSTZ'})

            ##reactions
            reactions=generics[0].find('span',{'class':'_2plerxB_dRyqlQe66M6Ay3'}).text
            
            ##posts
            posts= generics[1].find('span',{'class':'_2plerxB_dRyqlQe66M6Ay3'}).text

            ##replies
            replies= generics[2].find('span',{'class':'_2plerxB_dRyqlQe66M6Ay3'}).text

            ##lastActive
            lastActive= soup2.find_all('div', {'class':'_3Bw5KIqCTKiMsOeIV9fSTZ'})[0].find('span',{'class':'_2plerxB_dRyqlQe66M6Ay3'}).text

            temp_list=[thread_title, username, profile_url,reactions,posts,replies, lastActive]
            full_list.append(temp_list)
  
df = pd.DataFrame(full_list,columns=("Thread_Title","Username","Profile_url","Reactions","Posts","Replies", "LastActive"))
df.to_csv('/content/sample_data/master_df_aus1_chlamydia.csv', index= False)


Hicks
https://forums.au.reachout.com/t5/user/viewprofilepage/user-id/6676
Sophie-RO
https://forums.au.reachout.com/t5/user/viewprofilepage/user-id/52
Chessca_H
https://forums.au.reachout.com/t5/user/viewprofilepage/user-id/4265
AnotherMishap
https://forums.au.reachout.com/t5/user/viewprofilepage/user-id/6291
Ben-RO
https://forums.au.reachout.com/t5/user/viewprofilepage/user-id/7149


In [66]:
df= pd.read_csv('/content/sample_data/master_df_aus1_chlamydia.csv')

In [67]:
df.head()

,Thread_Title,Username,Profile_url,Reactions,Posts,Replies,LastActive
0,I'm hiding my relationship and so is he is tha...,Hicks,https://forums.au.reachout.com/t5/user/viewpro...,14,14,47,14th March
1,Sex: myth busters!,Sophie-RO,https://forums.au.reachout.com/t5/user/viewpro...,"3,381",200,"3,261",14th October
2,[Special Guest] Supporting Survivors Part 2: L...,Chessca_H,https://forums.au.reachout.com/t5/user/viewpro...,"1,481",17,"2,129",8th December
3,I don't know what to do anymore.,AnotherMishap,https://forums.au.reachout.com/t5/user/viewpro...,1,1,1,22nd October
4,[Special Guest] Sexual Health with Nurse Netti...,Ben-RO,https://forums.au.reachout.com/t5/user/viewpro...,"4,612",196,"6,017",3rd July


In [68]:
full_final_list=[]
for index, row in df.iterrows():
    temp_list1 =[]
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    
    profile_url= row['Profile_url']
    print(profile_url)

    #crawling through profile page
    wd = webdriver.Chrome('chromedriver',options=options)
    r2=wd.get(profile_url)
    r3=wd.page_source#)  # results
    soup3 = BeautifulSoup(r3, 'html.parser')
    
    #
    threads_on_page= soup3.find_all('div', {'class':'ro-feed-post__body'})#[].text
    print(threads_on_page)
    print("len(threads_on_page)", len(threads_on_page))

    for i in range(len(threads_on_page)):
      response_text= threads_on_page[i].text

      if 'read more' in str(threads_on_page[i].text):
        new_url= soup3.find_all('div',{'class':'ro-feed-post__body'})[i].find('a')['href']
        profile_url_new = new_url
        print("profile_url_new \n", profile_url_new)
        
        wd = webdriver.Chrome('chromedriver',options=options)
        r4=wd.get(profile_url_new)
        r5=wd.page_source#)  # results
        soup4 = BeautifulSoup(r5, 'html.parser')

        response_text= soup4.find_all('div',{'class':'ro-topic__body'})[0]
        response_time= soup4.find_all('span',{'class':'ro-topic__author__post-time'})[0].text

      else:
        response_text= response_text
        response_time= soup3.find_all('div',{'class':'ro-feed-post__author__post-details'})[i].find('span').text
        print(response_time)

      temp_list1= [row['Thread_Title'], row['Username'], row['Profile_url'], row['Reactions'], row['Posts'], row['Replies'], row['LastActive'], response_text, response_time]
      full_final_list.append(temp_list1)

df_final = pd.DataFrame(full_final_list,columns=("Thread_Title","Username","Profile_url","Reactions","Posts","Replies", "LastActive", "Response_text", "Response_time"))
df_final.to_csv('/content/sample_data/master_df_aus1_chlamydia_final.csv', index= False)

 

https://forums.au.reachout.com/t5/user/viewprofilepage/user-id/6676
[<div class="ro-feed-post__body"><p>Idk I’ve seen him on and off for 4 months and he’s suggested that we actually go ahead and have sex either unprotected or protected but isn’t that wrong? I’ve already told him I have herpes all I’m really interested in is teasing him and just talking really though. He’s even suggested not charging m... <a classname="ro-feed-post__read-more" href="https://forums.au.reachout.com/t5/Heavy-feels/I-ve-been-seeing-an-escort-for-4-months/td-p/423075">read more</a></p></div>, <div class="ro-feed-post__body"><p>We used to only have one dog and that wasn't bad as we could keep the back door open all the time and he wouldn't escape; but since we got another dog a few years ago she's only been getting pregnant, escaping and constantly barking. I feel like I can't live at home anymore as the dogs keep going to... <a classname="ro-feed-post__read-more" href="https://forums.au.reachout.com/t5/Takin